# GeneralAnalysisDemo

This tests the new generalized scanning framework "miniScatterScanner.py", as well as the loading of a TTree with all the data and doing a rudimentary analysis of it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

#colorCycle = plt.rcParams['axes.prop_cycle']
plt.rcParams['figure.max_open_warning'] = 100

In [3]:
import miniScatterDriver
import miniScatterScanner

Welcome to JupyROOT 6.14/02


/usr/lib64/python3.6/site-packages/ROOT.py:461: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return _orig_ihook( name, *args, **kwds )


In [4]:
#Basic parameters
NUM_THREADS = 16
QUIET = False
TRYLOAD = False #Try to load already existing data instead of recomputing?

baseSimSetup = {}
baseSimSetup["PHYS"] = "QGSP_BERT__SS"
baseSimSetup["BEAM"] = "e-"

baseSimSetup["THICK"] = 15 #Target thickness [mm]

baseSimSetup["ENERGY"] = 215.0 #[MeV]
EPS   = 3.0 #[um]
BETA  = 1.4 #[m]
ALPHA = 0.0
baseSimSetup["COVAR"] = (EPS,BETA,ALPHA)

baseSimSetup["ZOFFSET"] = -baseSimSetup["THICK"]
baseSimSetup["ZOFFSET_BACKTRACK"]=True

baseSimSetup["DIST"] = 1000.0 #[mm]

baseSimSetup["QUICKMODE"] = True #Skip verification plots

#The scan
#press = np.logspace(np.log10(0.5),np.log10(100), 30) #[mbar]
#COMMENT="LOGSPACE"

#press = np.linspace(0.5,100,30)
press = np.linspace(0.5,100,3)
COMMENT = "LINSPACE"

print("press =", press, "[mbar], COMMENT={}".format(COMMENT))

#Lowest energy particles to consider for analysis
E_CUTOFF = baseSimSetup["THICK"]*0.9;


press = [  0.5   50.25 100.  ] [mbar], COMMENT=LINSPACE


In [5]:
baseSimSetup_Ar = baseSimSetup.copy()
baseSimSetup["MAT"] = "Ar"
baseSimSetup["N"] = 1000

def countParticleTypes(datafile):
    out = {}
    out["n_em"] = 0.0
    out["n_ep"] = 0.0
    
    for hit in datafile.TargetExit:
        if hit.E > E_CUTOFF:
                if hit.PDG == 11:
                    out["n_em"] += 1
                elif hit.PDG == 11:
                    out["n_ep"] += 1
    
    #for hit in datafile.TrackerHits:
    #    print (hit)
    
    return out

(eps_x_Ar,eps_y_Ar, beta_x_Ar,beta_y_Ar, alpha_x_Ar,alpha_y_Ar, sigma_x_Ar,sigma_y_Ar, analysisOutput) = \
    miniScatterScanner.ScanMiniScatter("PRESS",press,baseSimSetup, \
                                       NUM_THREADS=NUM_THREADS,tryLoad=TRYLOAD,COMMENT=COMMENT, QUIET=QUIET, \
                                       detailedAnalysisRoutine=countParticleTypes,detailedAnalysisRoutine_names=["n_em","n_ep"])

#plt.figure()
#plt.semilogx(press,eps_x_Ar)
#plt.semilogx(press,eps_y_Ar)

LoadFile filename and status: 'SaveSim_BEAM=e---COVAR=(3.0, 1.4, 0.0)--DIST=1000.0--ENERGY=215.0--MAT=Ar--N=1000--PHYS=QGSP_BERT__SS--QUICKMODE=True--THICK=15--ZOFFSET=-15--ZOFFSET_BACKTRACK=True--SCANVAR=PRESS=[0.5,50.25,100.0]--COMMENT=LINSPACE.h5' False
PRESS = 0.5 (1/3)
Running command line: './MiniScatter -t 15 -m Ar::0.5 -d 1000.0 -p QGSP_BERT__SS -n 1000 -e 215.0 -b e- -z *-15 -c 3.0:1.4:0.0 -s 1 -f output_PRESS=0.5 -q'
PRESS = 50.25 (2/3)
Running command line: './MiniScatter -t 15 -m Ar::50.25 -d 1000.0 -p QGSP_BERT__SS -n 1000 -e 215.0 -b e- -z *-15 -c 3.0:1.4:0.0 -s 2 -f output_PRESS=50.25 -q'
PRESS = 100.0 (3/3)
Running command line: './MiniScatter -t 15 -m Ar::100.0 -d 1000.0 -p QGSP_BERT__SS -n 1000 -e 215.0 -b e- -z *-15 -c 3.0:1.4:0.0 -s 3 -f output_PRESS=100.0 -q'
Done!
X : 2.9919676986849106 1.4355181855272048 0.02319386587972064
Y : 3.1492388895857344 1.3711964875156835 -0.029115340654662585
Done!
X : 2.9919676986849106 1.4355181855272048 0.02319386587972064
Y : 3.149

In [6]:
analysisOutput

{'n_em': array([1000., 1001., 1001.]), 'n_ep': array([0., 0., 0.])}